In [1]:
import pandas as pd

df = pd.read_csv('./datasets/hewiki/hebrew_nikud_dataset.csv')

In [2]:
# Display the first few rows of the dataframe
print(df.head())

# Show basic info about the dataframe
print(df.info())

# Show summary statistics for the length of text and article_length columns
df['text_length'] = df['text'].apply(len)
print(df[['text_length', 'article_length']].describe())

# Check for missing values
print(df.isnull().sum())

# Display the distribution of nikud_mask lengths
df['nikud_mask_length'] = df['nikud_mask'].apply(lambda x: len(x.split()))
print(df['nikud_mask_length'].value_counts().head())

                                                text  \
0  מָתֵמָטִיקָה היא תחום דעת העוסק במושגים כגון כ...   
1  תחום מחקר נוסף הוא גודל, אשר מוביל לעוצמות ולת...   
2       ריגורוזיוּת היא מרכיב יסודי בכל הוכחה מתמטית   
3  חישובים בהיקף גדול קשים לאימות, ולכן יש הטועני...   
4  אַקְסיּוֹמָה, אמיתה, או הנחת יסוד (בכתיב ארכאי...   

                                          nikud_mask article_title  \
0  [0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...       מתמטיקה   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...       מתמטיקה   
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...       מתמטיקה   
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...       מתמטיקה   
4  [0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...       אקסיומה   

   article_length  
0            3577  
1            3577  
2            3577  
3            3577  
4             505  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118040 entries, 0 to 118039
Data columns (total 4 columns):
 #   Column 

In [3]:
import re

NIKUD_PATTERN = re.compile(
    '['
    '\u05B0'  # sheva
    '\u05B1'  # hataf segol
    '\u05B2'  # hataf patah
    '\u05B3'  # hataf qamats
    '\u05B4'  # hiriq
    '\u05B5'  # tsere
    '\u05B6'  # segol
    '\u05B7'  # patah
    '\u05B8'  # qamats
    '\u05B9'  # holam
    '\u05BB'  # qubuts
    '\u05BC'  # dagesh or mapiq
    '\u05BD'  # meteg
    '\u05BF'  # rafe (rare)
    '\u05C1'  # shin dot
    '\u05C2'  # sin dot
    '\u05C7'  # qamats qatan
    ']'
)

In [4]:
# Check if each row contains at least one word with nikud using NIKUD_PATTERN
def has_word_with_nikud(text):
    return any(NIKUD_PATTERN.search(word) for word in text.split())

# Apply the function to the 'text' column and check if all rows satisfy the condition
all_have_nikud = df['text'].apply(has_word_with_nikud).all()
print("All rows contain at least one word with nikud:", all_have_nikud)

All rows contain at least one word with nikud: True


In [5]:
# Find rows where the nikud_mask contains only one to three 1s in a row (consecutive)
def has_1s_run(mask, min_run=1, max_run=3):
    mask_str = ''.join(str(x) for x in mask)
    # Check for runs of 1s of length 1, 2, or 3
    for run_length in range(max_run, min_run - 1, -1):
        # print(f"Checking for run of {run_length} 1s in mask: {mask_str}")
        i = mask_str.find(f"{'1' * run_length}")
        if i != -1:
            # print(f"Found run of {run_length} 1s in mask: {mask_str} at position {i}")
            # Check if the run is isolated by at least 3 zeros
            if (i == 0 or mask_str[i - 1] == '0') and (i + run_length == len(mask_str) or mask_str[i + run_length] == '0') and \
               (i <= 1 or mask_str[i - 2] == '0') and (i + run_length + 1 >= len(mask_str) or mask_str[i + run_length + 1] == '0'):
                return True
    return False

In [6]:
filtered_rows = df.sample(5)
filtered_rows.iloc[0]['nikud_mask']


'[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]'

In [7]:
import ast

filtered_df = df[df['nikud_mask'].apply(lambda x: has_1s_run(ast.literal_eval(x)) if isinstance(x, str) else has_1s_run(x))]
filtered_df.shape

(81995, 6)

In [8]:
filtered_df.head()

,text,nikud_mask,article_title,article_length,text_length,nikud_mask_length
1,"תחום מחקר נוסף הוא גודל, אשר מוביל לעוצמות ולת...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",מתמטיקה,3577,141,141
2,ריגורוזיוּת היא מרכיב יסודי בכל הוכחה מתמטית,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",מתמטיקה,3577,44,45
3,"חישובים בהיקף גדול קשים לאימות, ולכן יש הטועני...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",מתמטיקה,3577,85,86
5,"תורת האֶתֶר גרסה שיש חומר הממלא את כל המרחב, ו...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, ...",אקסיומה,505,70,70
6,בניסוי מייקלסון-מורלי נמדדה מהירות הקפת כדור ה...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",אקסיומה,505,151,151


In [9]:
filtered_df.to_csv('./datasets/hewiki/hebrew_nikud_dataset_filtered.csv', index=False)